**老师图 = 基于分子表示得到的概率图 $Q^\*$**，**学生图 = 基于坐标 $Y$ 得到的概率图 $Q(Y)$**，用**交叉熵 CE(Q\*, Q(Y))** 做监督，只更新 $Y$（不加物理约束）。

下面给出一份**PyTorch 伪代码**（可直接改成可运行代码）。

---

# 学习模型与可学习参数

* **输入**

  * 分子表示得到的两两“距离”或相似度矩阵 $D_{\text{rep}}$（或你已有的 P 矩阵也行）。
  * 可选的拓扑掩码 `mask_pairs`（如只监督 hop≤3 的成对关系）。
* **核函数**（与 UMAP 类似）

  $$
  Q_{ij}(\cdot)=\frac{1}{1+a\,d_{ij}^{2b}},\quad a>0,b>0
  $$
* **老师图**

  $$
  Q^\*=\;Q(D_{\text{rep}})\quad\text{（把分子表示的距离喂进同一核函数得到概率）}
  $$

  （若你已有用 smooth-k/fuzzy-union 得到的 $P$，也可以直接把 $Q^\*=P$。）
* **学生图**

  $$
  Q(Y)=Q\!\big(\,d_{ij}(Y)=\|y_i-y_j\|_2\,\big)
  $$
* **目标函数（监督 CE，仅更新 $Y$**）

  $$
  \mathcal{L}=\mathrm{CE}(Q^\*,Q(Y))
  =-\sum_{i\neq j}\big(Q^\*_{ij}\log Q_{ij}(Y)+(1-Q^\*_{ij})\log(1-Q_{ij}(Y))\big)
  $$
* **可学习参数**

  * 必选：$Y\in\mathbb{R}^{N\times 3}$（作为 `nn.Parameter`）。
  * 可选（保持简单可先固定）：全局 $a,b$；或每行带宽 $\sigma_i$（进阶）。

---

# PyTorch 伪代码（不含物理约束）

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

# ---------------------------
# 0) 准备与实用函数
# ---------------------------
def center_and_rescale_torch(Y, target_rms=1.0, eps=1e-8):
    # 去中心 + 统一尺度（刚体不变 & 稳定优化）
    Yc = Y - Y.mean(dim=0, keepdim=True)
    rms = torch.sqrt((Yc**2).mean())
    scale = target_rms / (rms + eps)
    return Yc * scale

def Q_from_dist(D, a, b, eps=1e-12):
    # 输入任意距离矩阵 D (N,N)；输出概率矩阵 Q in (0,1)
    Q = 1.0 / (1.0 + a * (D.clamp_min(eps) ** (2.0 * b)))
    Q = Q.clamp(1e-12, 1.0 - 1e-12)
    return Q

def pairwise_dist_from_Y(Y, eps=1e-12):
    # 使用 torch.cdist 计算两两欧氏距离
    D = torch.cdist(Y, Y, p=2)
    return D.clamp_min(eps)

def ce_loss(Q_star, Q_pred, mask=None):
    # 交叉熵（成对），可选 mask（如去掉对角、或只监督 hop<=3）
    if mask is not None:
        Qs = Q_star[mask]
        Qp = Q_pred[mask]
    else:
        Qs = Q_star
        Qp = Q_pred
    return -(Qs * torch.log(Qp) + (1 - Qs) * torch.log(1 - Qp)).mean()

# （可选）把 min_dist 转成 a,b；简单起见可先固定 a,b
def find_ab_from_min_dist(min_dist=0.5, spread=3.0, device="cpu"):
    # 简单网格近似；项目里可预先离线求好常数
    xs = torch.linspace(0, spread, 256, device=device)
    target = torch.where(xs <= min_dist, torch.ones_like(xs), torch.exp(-(xs - min_dist)))
    best = (torch.tensor(1.0, device=device), torch.tensor(1.0, device=device), 1e9)
    for a_ in torch.linspace(0.5, 3.0, 26, device=device):
        for b_ in torch.linspace(0.3, 1.6, 27, device=device):
            pred = 1.0 / (1.0 + a_ * (xs ** (2.0 * b_)))
            err = torch.mean((pred - target) ** 2).item()
            if err < best[2]:
                best = (a_, b_, err)
    return float(best[0]), float(best[1])

# ---------------------------
# 1) 构造老师图 Q_star
# ---------------------------
def build_teacher_Q_from_rep(D_rep, a, b, mask_pairs=None, device="cpu"):
    """
    D_rep: 由分子表示得到的两两距离/不相似度 (N,N), torch.Tensor
           若你手头是“相似度 S”，可先转 D_rep = f(S) 再用本函数。
    a,b  : 核函数参数（可固定）
    mask_pairs: bool mask (N,N)（如只监督 hop<=3，且去对角）
    """
    Q_star = Q_from_dist(D_rep.to(device), a, b)
    # 对角不监督
    N = Q_star.size(0)
    eye = torch.eye(N, dtype=torch.bool, device=device)
    base_mask = ~eye
    if mask_pairs is not None:
        base_mask = base_mask & mask_pairs.to(device)
    return Q_star, base_mask

# ---------------------------
# 2) 训练主循环：只学 Y
# ---------------------------
def fit_Y_by_supervised_CE(D_rep,
                           N_atoms,
                           a=1.6, b=0.8,         # 或使用 find_ab_from_min_dist() 求
                           mask_pairs=None,       # 例如 (hop<=3)
                           lr=1e-2, epochs=1000,
                           device="cpu",
                           max_step=0.2):
    """
    返回学到的 Y (N,3)
    """
    # 老师图
    Q_star, mask = build_teacher_Q_from_rep(D_rep, a, b, mask_pairs, device=device)

    # 初始化 Y（可用随机 / PCA / 你的初始化）
    Y = nn.Parameter(torch.randn(N_atoms, 3, device=device) * 0.1)
    opt = torch.optim.Adam([Y], lr=lr)

    for t in range(epochs):
        # 刚体规范化（可放在更新后；此处放前/后都可）
        with torch.no_grad():
            Y.data = center_and_rescale_torch(Y.data, target_rms=1.0)

        # 前向：Q(Y)
        D_Y = pairwise_dist_from_Y(Y)
        Q_Y = Q_from_dist(D_Y, a, b)

        # CE 损失（只更新 Y）
        loss = ce_loss(Q_star, Q_Y, mask=mask)

        opt.zero_grad()
        loss.backward()

        # （可选）梯度或位移裁剪，避免过大步导致数值抖动
        with torch.no_grad():
            # 基于梯度构造本步位移并裁剪每原子步长
            step = -lr * Y.grad
            # 逐原子 trust-region
            step_norm = torch.norm(step, dim=1, keepdim=True) + 1e-12
            step = step * torch.minimum(torch.ones_like(step_norm),
                                        torch.tensor(max_step, device=device) / step_norm)
            Y.data += step

        if (t % 50) == 0:
            print(f"[{t:04d}] CE_sup={loss.item():.6f}")

    # 结束前做一次规范化
    with torch.no_grad():
        Y.data = center_and_rescale_torch(Y.data, target_rms=1.0)
    return Y.detach()

# ---------------------------
# 3) 用法示例
# ---------------------------
# 假设你已经有 N、和表示得到的 D_rep (N,N)：
# device = "cuda" if torch.cuda.is_available() else "cpu"
# a, b = find_ab_from_min_dist(min_dist=0.5, device=device)  # 或直接给定 a,b
# Y_hat = fit_Y_by_supervised_CE(D_rep=torch.tensor(D_rep, dtype=torch.float32, device=device),
#                                N_atoms=D_rep.shape[0],
#                                a=a, b=b,
#                                mask_pairs=torch.tensor(hop<=3),  # 可选；无则传 None
#                                lr=0.01, epochs=1000, device=device)
```

---

## 说明与小贴士

* **为什么这样做有优势？**
  仅用 CE 对齐图：

  1. 对平移/旋转天然不敏感（只看距离），训练稳；
  2. “吸引 + 排斥”两种信号同时存在，防塌缩；
  3. 若用同一核函数监督**全矩阵**且 CE→0，则两套距离矩阵一致 ⇒ $Y$ 与老师构象一致（刚体/镜像自由度除外）。

* **老师图怎么来？**
  上面使用 `Q_from_dist(D_rep, a, b)`；如果你已有 UMAP 风格的 $P$（smooth-k + fuzzy union），直接把 `Q_star = P` 也是可以的（目标只是一个 \[0,1] 概率图）。

* **可选扩展（保持“只学 Y”的前提下）**

  * 固定 $a,b$ 够用；若想更贴合当前尺度，可每隔 30 轮用当前正对距离的 60% 分位数**微调 min\_dist**，并重新估 $a,b$。
  * 如果只监督 hop≤3，远程自由度较多；但这一版你明确“先不加物理”，完全 OK。

把上面的伪代码稍作变量替换（把你的 `D1` 或表示距离矩阵塞给 `D_rep`）即可跑起来。需要我帮你把它**融入你现有脚本**（保留你函数名/记录/可视化）的话，直接把变量名和初始化部分贴我一份就行。
